<a href="https://colab.research.google.com/github/davidfague/Model_Reduction_Methods/blob/main/NMDASimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation for generating data for dendritic spike analysis
##Recorded Currents:

Na,K,Ca,ih,...

## synapse distribution:

10,000 random. will update to include more realistic algorithm

#### Download modules from Github

In [1]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 32.6 MB/s eta 0:00:00


In [2]:
!pip install neuron_reduce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!git clone https://github.com/davidfague/Model_Reduction_Methods.git

Cloning into 'Model_Reduction_Methods'...
remote: Enumerating objects: 749, done.
remote: Counting objects: 100% (435/435), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 749 (delta 260), reused 303 (delta 174), pack-reused 314
Receiving objects: 100% (749/749), 5.40 MiB | 14.13 MiB/s, done.
Resolving deltas: 100% (411/411), done.


In [4]:
%cd Model_Reduction_Methods/

#import reduction and expansion functions
from test_neuron_reduce.subtree_reductor_func import subtree_reductor
from cable_expander_func import cable_expander

#import recording functions
from stylized_module.recorder import Recorder

#import analysis functions
from utils import make_seg_df,generate_stylized_geometry,make_reduced_seg_df,plot_morphology,check_connectivity,generate_reduced_cell_seg_coords, create_seg_var_report


# from modeling_module.synapses import Synapse, Listed_Synapse
from modeling_module.cell_model import cell_model

import pandas as pd

/content/Model_Reduction_Methods


In [5]:
%cd expand_example

/content/Model_Reduction_Methods/expand_example


In [6]:
# compile the mod files
!nrnivmodl mod

/content/Model_Reduction_Methods/expand_example
Mod files: "mod/mod/CaDynamics_E2.mod" "mod/mod/Ca_HVA.mod" "mod/mod/Ca_LVAst.mod" "mod/mod/epsp.mod" "mod/mod/Ih.mod" "mod/mod/Im.mod" "mod/mod/K_Pst.mod" "mod/mod/K_Tst.mod" "mod/mod/Nap_Et2.mod" "mod/mod/NaTa_t.mod" "mod/mod/NaTs2_t.mod" "mod/mod/SK_E2.mod" "mod/mod/SKv3_1.mod"

Creating 'x86_64' directory for .o files.

 -> NMODL ../mod/Ca_LVAst.mod
 -> Compiling mod_func.cpp
 -> NMODL ../mod/Ca_HVA.mod
 -> NMODL ../mod/CaDynamics_E2.mod
Translating CaDynamics_E2.mod into /content/Model_Reduction_Methods/expand_example/x86_64/CaDynamics_E2.c
Translating Ca_LVAst.mod into /content/Model_Reduction_Methods/expand_example/x86_64/Ca_LVAst.c
Thread Safe
Translating Ca_HVA.mod into /content/Model_Reduction_Methods/expand_example/x86_64/Ca_HVA.c
Thread Safe
Thread Safe
 -> NMODL ../mod/epsp.mod
 -> NMODL ../mod/Im.mod
 -> NMODL ../mod/Ih.mod
Translating epsp.mod into /content/Model_Reduction_Methods/expand_example/x86_64/epsp.c
Thread Safe
 -

## Setup smiulation parameters

In [7]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import ipywidgets as widgets
from neuron import h
from scipy import signal
from IPython.display import display, clear_output
from ipywidgets import interactive_output, HBox, VBox, Label, Layout

from __future__ import division
from neuron import gui,h
import numpy as np
import time
import matplotlib.pyplot as plt

%matplotlib inline


h.load_file('stdrun.hoc')
# h.nrn_load_dll(paths.COMPILED_LIBRARY_REDUCED_ORDER)  # choose the set of mechanisms
h.nrn_load_dll('./x86_64/.libs/libnrnmech.so')


1.0

### Create a cell with reduced morphology

In [8]:
%ls

cell1.asc  example_expand.py  L5PCtemplate.hoc  x86_64/
Cell.hoc   L5PCbiophys3.hoc   mod/


In [9]:

h.load_file('L5PCbiophys3.hoc') # load membrane biophysics
h.load_file("import3d.hoc") #load 3d morphology


# Create a cell object
h.load_file('L5PCtemplate.hoc') # load template for generating object
complex_cell = h.L5PCtemplate('cell1.asc') # generate object

#specify some parameters
h.celsius = 37
h.v_init = complex_cell.soma[0].e_pas

#Add synapses to the complex model
synapses_list, netstims_list, netcons_list, randoms_list = [], [], [] ,[]

all_segments = [i for j in map(list,list(complex_cell.apical)) for i in j] + [i for j in map(list,list(complex_cell.basal)) for i in j]
len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in all_segments])
rnd = np.random.RandomState(10)
for i in range(10000):
    seg_for_synapse = rnd.choice(all_segments,   p=len_per_segment/sum(len_per_segment)) #choose a random segment with probability based on the length of segment
    synapses_list.append(h.Exp2Syn(seg_for_synapse))
    if rnd.uniform()<0.85: # 85% synapses are excitatory
        e_syn, tau1, tau2, spike_interval, syn_weight = 0, 0.3, 1.8,  1000/2.5, 0.0016
    else: #inhibitory case
        e_syn, tau1, tau2, spike_interval, syn_weight = -86, 1,   8,   1000/15.0, 0.0008
    #set synaptic varibales
    synapses_list[i].e, synapses_list[i].tau1, synapses_list[i].tau2 = e_syn, tau1, tau2
    #set netstim variables
    netstims_list.append(h.NetStim())
    netstims_list[i].interval, netstims_list[i].number, netstims_list[i].start, netstims_list[i].noise = spike_interval, 9e9, 100, 1
    #set random
    randoms_list.append(h.Random())
    randoms_list[i].Random123(i)
    randoms_list[i].negexp(1)
    netstims_list[i].noiseFromRandom(randoms_list[i])       
    #set netcon varibales 
    netcons_list.append(h.NetCon(netstims_list[i], synapses_list[i] ))
    netcons_list[i].delay, netcons_list[i].weight[0] = 0, syn_weight

In [10]:
#reduce each dendritic subtree to a single cable
reduced_cell, synapses_list, netcons_list, txt = subtree_reductor(complex_cell, synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

In [11]:
#check synapses_list with netcons_list
for netcon in netcons_list:
  syn=netcon.syn()
  if syn not in synapses_list:
    print(syn, netcon)

In [17]:
indices=[109,122,130,309,360,364,446,629,290,791,851,930,951,967,975,981,1047,1104]
for index in indices:
  print(netcons_list[index], netcons_list[index].syn(),netcons_list[index].syn().get_segment())



NetCon[109] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[122] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[130] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[309] Exp2Syn[309] model[0].apic[0](0.289474)
NetCon[360] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[364] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[446] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[629] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[290] Exp2Syn[47] model[0].apic[0](0.657895)
NetCon[791] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[851] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[930] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[951] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[967] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[975] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[981] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[1047] Exp2Syn[109] model[0].apic[0](0.289474)
NetCon[1104] Exp2Syn[309] model[0].apic[0](0.289474)


In [ ]:
#expand cables to idealized dendritic trees
sections_to_expand = [reduced_cell.hoc_model.apic[0]] # expand apical cylinder
furcations_x=[0.289004] #chose using the location of the mapped nexus branching segment
nbranches=[4]
reduced_dendritic_cell, synapses_list, netcons_list, txt = cable_expander(reduced_cell, sections_to_expand, furcations_x, nbranches, 
                                                                          synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

In [ ]:
#check synapses_list with netcons_list
for netcon in netcons_list:
  syn=netcon.syn()
  if syn not in synapses_list:
    print(syn, netcon)

In [ ]:
# # try branching the branches - does not work yet
# sections_to_expand = [reduced_dendritic_cell.apic[1],reduced_dendritic_cell.apic[2],reduced_dendritic_cell.apic[3],reduced_dendritic_cell.apic[4]]
# furcations_x=[0.50,0.50,0.50,0.50]
# nbranches=[4,4,4,4]
# reduced_dendritic_cell, synapses_list, netcons_list, txt = cable_expander(reduced_dendritic_cell, sections_to_expand, furcations_x, nbranches, 
#                                                                           synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

In [ ]:
#check seg mapping
# for i in txt:
#   print(i,"was mapped to",txt[i])

In [ ]:
#use defined cell_model python class for generating 3d coordinates, recording ECP, 'book-keeping' etc...
import random
random.seed(2)
cell = cell_model(reduced_dendritic_cell,synapses_list=synapses_list,netcons_list=netcons_list,spike_threshold = 10)
# cell._nbranch=4

In [ ]:
# for sec in cell.all:
#   for seg in sec:
#     for pp in seg.point_processes():
#       print(pp)

In [ ]:
# #potentially deleted synapses?
# for netcon in netcons_list:
#   if netcon.syn() not in synapses_list:
#     print(netcon)
#     print(netcon.syn() in synapses_list)
#     print(netcon.syn())

In [ ]:
# for syn in synapses_list:
#   print(syn,syn.get_segment().point_processes())

In [ ]:
# #create second original model

# original_cell = h.L5PCtemplate('cell1.asc')
# synapses_list, netstims_list, netcons_list, randoms_list = [], [], [] ,[]

# all_segments = [i for j in map(list,list(original_cell.apical)) for i in j] + [i for j in map(list,list(original_cell.basal)) for i in j]
# len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in all_segments])
# rnd = np.random.RandomState(10)
# for i in range(10000):
#     seg_for_synapse = rnd.choice(all_segments,   p=len_per_segment/sum(len_per_segment)) #choose a random segment with probability based on the length of segment
#     synapses_list.append(h.Exp2Syn(seg_for_synapse))
#     if rnd.uniform()<0.85: # 85% synapses are excitatory
#         e_syn, tau1, tau2, spike_interval, syn_weight = 0, 0.3, 1.8,  1000/2.5, 0.0016
#     else: #inhibitory case
#         e_syn, tau1, tau2, spike_interval, syn_weight = -86, 1,   8,   1000/15.0, 0.0008
#     #set synaptic varibales
#     synapses_list[i].e, synapses_list[i].tau1, synapses_list[i].tau2 = e_syn, tau1, tau2
#     #set netstim variables
#     netstims_list.append(h.NetStim())
#     netstims_list[i].interval, netstims_list[i].number, netstims_list[i].start, netstims_list[i].noise = spike_interval, 9e9, 100, 1
#     #set random
#     randoms_list.append(h.Random())
#     randoms_list[i].Random123(i)
#     randoms_list[i].negexp(1)
#     netstims_list[i].noiseFromRandom(randoms_list[i])       
#     #set netcon varibales 
#     netcons_list.append(h.NetCon(netstims_list[i], synapses_list[i] ))
#     netcons_list[i].delay, netcons_list[i].weight[0] = 0, syn_weight
# original_model = cell_model(original_cell,gen_3d=False,spike_threshold = 10)
# print(synapses_list)

In [ ]:
make_reduced_seg_df(cell,"segments_expanded.csv") #need to improve make_reduced_seg_df
expanded_segments_df=pd.read_csv("segments_expanded.csv")
plot_morphology(expanded_segments_df,"expanded_morphology.svg")

# change to complex cell
# make_reduced_seg_df(cell,"segments_expanded.csv") #need to improve make_reduced_seg_df
# expanded_segments_df=pd.read_csv("segments_expanded.csv")
# plot_morphology(expanded_segments_df,"expanded_morphology.svg")

In [ ]:
# dir(cell)

In [ ]:
import os

RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !pip install neuron==8.0.0 &> /dev/null
    os.chdir('/content')
    if not os.path.isdir('Stylized-Single-Cell-and-Extracellular-Potential'):
        !git clone https://github.com/chenziao/Stylized-Single-Cell-and-Extracellular-Potential.git &> /dev/null 
    os.chdir('Stylized-Single-Cell-and-Extracellular-Potential')
    %ls

In [ ]:
from typing import Union, List, Tuple


class Recorder(object):
    """A module for recording variables"""

    def __init__(self, obj_list: Union[object, List[object], Tuple[object], np.ndarray], var_name: str = 'v') -> None:
        """
        obj_list: list of (or a single) target objects
        var_name: string of variable to be recorded
        """
        self.single = not isinstance(obj_list, (list, tuple, np.ndarray))
        self.obj_list = obj_list
        self.var_name = var_name
        self.vectors = None
        self.setup_recorder()

    def setup_recorder(self) -> None:
        size = [round(h.tstop / h.dt) + 1] if hasattr(h, 'tstop') else []
        attr_name = '_ref_' + self.var_name
        if self.single:
            self.vectors = h.Vector(*size).record(getattr(self.obj_list, attr_name))
        else:
            self.vectors = [h.Vector(*size).record(getattr(obj, attr_name)) for obj in self.obj_list]

    def as_numpy(self, copy: bool = True) -> np.ndarray:
        """
        Return a numpy 2d-array of recording, n objects-by-time
        Return a 1d-array if a single object is being recorded
        """
        if self.single:
            x = self.vectors.as_numpy()
        else:
            x = np.array([v.as_numpy() for v in self.vectors])
        if copy:
            x = x.copy()
        return x

In [ ]:
#insert unused channels for recorder
for sec in cell.all:
  if not hasattr(sec(0.5),'gNaTa_t_NaTa_t'):
    sec.insert('NaTa_t')
    for seg in sec:
      seg.NaTa_t.gNaTa_tbar=0
    print(sec)

In [ ]:
#insert unused channels for recorder
for sec in cell.all:
  if not hasattr(sec(0.5),'ica_Ca_LVAst'):
    sec.insert('Ca_LVAst')
    for seg in sec:
      seg.Ca_LVAst.gCa_LVAstbar=0
    print(sec)

In [ ]:
#insert unused channels for recorder
for sec in cell.all:
  if not hasattr(sec(0.5),'ica_Ca_HVA'):
    sec.insert('Ca_HVA')
    for seg in sec:
      seg.Ca_HVA.gCa_HVAbar

In [ ]:
#insert unused channels for recorder
for sec in cell.all:
  if not hasattr(sec(0.5),'ihcn_Ih'):
    sec.insert('Ih')
    for seg in sec:
      seg.Ih.gIhbar

In [ ]:
#define simulation parameters
# tstop = 6000  # ms simulation duration
tstop = 5000
dt = 0.1  # ms
h.tstop = tstop
h.dt = dt
h.steps_per_ms = 1/h.dt

#define parameters that may be used in notebook
nseg = len(cell.segments)
numTstep = int(tstop/dt)

#set up recorders
# gNaTa_T = Recorder(obj_list = original_model.segments, var_name = 'gNaTa_t_NaTa_t')
# ical = Recorder(obj_list = original_model.segments, var_name = 'ica_Ca_LVAst')
# icah = Recorder(obj_list = original_model.segments, var_name = 'ica_Ca_HVA')
# ih = Recorder(obj_list = original_model.segments, var_name = 'ihcn_Ih')
# Vm = Recorder(obj_list = original_model.segments)

# set up recorders
gNaTa_T = Recorder(obj_list = cell.segments, var_name = 'gNaTa_t_NaTa_t')
ical = Recorder(obj_list = cell.segments, var_name = 'ica_Ca_LVAst')
icah = Recorder(obj_list = cell.segments, var_name = 'ica_Ca_HVA')
ih = Recorder(obj_list = cell.segments, var_name = 'ihcn_Ih')
Vm = Recorder(obj_list = cell.segments)

In [ ]:
#Run simulation

timestart=time.time()
h.run()
timestop=time.time()
t = h.t # was t=h.t() but 'float' object not callable
elapsedtime=timestop-timestart
simtime=tstop/1000 #convert from ms to s
# totaltime= elapsedtime+elapseddeftime
print('It took',round(elapsedtime),'sec to run a',simtime,'sec simulation.')
# print('The total runtime was',round(totaltime),'sec')

In [ ]:
# get data from recorders
ih_data = ih.as_numpy()
gNaTa_T_data = gNaTa_T.as_numpy()
icah_data = icah.as_numpy()
ical_data = ical.as_numpy()
Vm = Vm.as_numpy()

In [ ]:
import h5py
def createsegtracereport(reportname,dataname):
  try:
    os.remove(reportname) # reportname was string " "
  except:
    x = 1

  f = h5py.File(reportname,'w') #create a file in the w (write) mode #reportname was string ' '
  v = f.create_dataset("report/biophysical/data", data = dataname)
  f.close()

In [ ]:
#calculate inmda
#since synapse list has combined synapses for computational efficiency, we must use NetCon list/unique spike trains to determine how many synapses were mapped
NetCon_per_seg=[0]*nseg
inh_NetCon_per_seg=[0]*nseg
exc_NetCon_per_seg=[0]*nseg

i_NMDA_bySeg= [[0] * (numTstep+1) ] * nseg # need to implement inmda recording

v_rest=-60 #choose v_rest for categorizing inh/exc synapses

#calculate number of synapses for each segment (may want to divide by segment length afterward to get synpatic density)
for netcon in netcons_list:
  syn=netcon.syn()
  if syn in synapses_list:
    syn_seg_id=cell.segments.index(netcon.syn().get_segment())
    if syn in cell.segments[syn_seg_id].point_processes():
      NetCon_per_seg[syn_seg_id]+=1 # get synapses per segment
      # NetCon_per_seg[syn_seg_id].append(netcon) # possible implementation if needing objects per segment
      if syn.e > v_rest:
        exc_NetCon_per_seg[syn_seg_id]+=1
        # exc_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
      else:
        inh_NetCon_per_seg[syn_seg_id]+=1
        # inh_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
    else:
      print("Warning: synapse not in designated segment's point processes")

  else:
    print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")

#extract inmda from each segment # can be adjusted for gaba synapses or alpha synapses # only paired nmda/ampa synapses have vec_list[1]
for synapse in cell.synapse:
  try:
    i_NMDA = np.array(synapse.rec_vec.vec_list[1])            #current = numpy array of NEURON Vector of current NMDA current at synapse j  
    seg = synapse.get_segment_id()                            #seg = the segment in which synapse j is located 
    #print('first try')
    
    try:
      i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA    
      #print('second try')                                           #Sum current over each segment
    except: 
      pass                                                                   #Except needed as some synpases do not have NMDA currrent and throw an error when called
  except:
    pass

In [ ]:

# numSyn = len(cell.synapse)

# excSynPerSeg = [0]*nseg
# inhSynPerSeg = [0]*nseg
# excSynPerSegL = [0]*nseg
# inhSynPerSegL = [0]*nseg
# SynParentSeg = []
# SourcePop = []
# SynType = []
# SynDist = []

# # i_NMDA_bySeg= [[0] * (numTstep+1) ] * nseg

#print(len(sim.cells[0].injection))
# for j in range(numSyn):
#   seg = cell.synapse[j].get_segment_id()
#   SynParentSeg.append(seg)
#   # SynType.append(AllSegType[seg])
#   # SynDist.append(AllSegDist[seg])

#   if(cell.synapse[j].syntype == 'exc'):
#     excSynPerSeg[seg] += 1
#     SourcePop.append('exc_stim')
#   else:
#     inhSynPerSeg[seg] += 1
#     SourcePop.append('dist_inh_stim')

#   try:
#     i_NMDA = np.array(cell.injection[j].rec_vec.vec_list[1])            #current = numpy array of NEURON Vector of current NMDA current at synapse j  
#     seg = cell.synapse[j].get_segment_id()                            #seg = the segment in which synapse j is located 
#     #print('first try')
    
#     try:
#       i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA    
#       #print('second try')                                           #Sum current over each segment
#     except: 
#       x = j                                                                     #Except needed as some synpases do not have NMDA currrent and throw an error when called
#   except:
#     x = j

In [ ]:
# i_NMDA_bySeg[0]

In [ ]:
#Should add "syn per seg" to seg data frame #also can try doing SynPerSeg/segL to have more of a density metric
def plotSynDensity(SynPerSeg):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(expanded_segments_df["Coord X"], expanded_segments_df["Coord Y"],c = SynPerSeg[0:nseg],cmap='jet',)
  plt.vlines(110,400,500)
  plt.text(0,450,'100 um')
  plt.hlines(400,110,210)
  plt.text(110,350,'100 um')
  plt.xticks([])
  plt.yticks([])
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Synapses per segment', rotation=270)

  plt.box(False)
  plt.savefig('exc_syns.svg')

def plotSynDensityNoSoma(SynPerSeg):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(expanded_segments_df["Coord X"][1:], expanded_segments_df["Coord Y"][1:],c = SynPerSeg[1:nseg],cmap='jet',)
  plt.vlines(110,400,500)
  plt.text(0,450,'100 um')
  plt.hlines(400,110,210)
  plt.text(110,350,'100 um')
  plt.xticks([])
  plt.yticks([])
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Synapses per segment', rotation=270)

  plt.box(False)
  plt.savefig('inh_syns.svg')

In [ ]:
plotSynDensity(exc_NetCon_per_seg) #len(cell.segments) #low number of segments makes plot look odd.

In [ ]:
plotSynDensityNoSoma(inh_NetCon_per_seg) # if soma has too many inh synapses the heatmap is not as interpretable

In [ ]:
i_NMDA_df = pd.DataFrame(i_NMDA_bySeg) * 1000

In [ ]:
output_folder_name=str(nbranches[0])+"nbranch_outputcontrol_"+str(int(len(cell.synapse)))+"nsyn"#+modelname  #include model name in output foler name (ex. original cell, reduced cell, expanded cell)
#create output folder
import os
if not os.path.exists(output_folder_name):
   os.makedirs(output_folder_name)

print(output_folder_name)
os.chdir(output_folder_name)

In [ ]:
#create output folder
import os
if not os.path.exists(output_folder_name):
   os.makedirs(output_folder_name)

In [ ]:
# output files for analysis in another notebook
createsegtracereport('v_report.h5', Vm.T)
createsegtracereport('Ca_HVA.ica_report.h5',icah_data.T)
createsegtracereport('Ca_LVAst.ica_report.h5',ical_data.T)
createsegtracereport('Ih.ihcn_report.h5',ih_data.T)
createsegtracereport('inmda_report.h5',i_NMDA_df.T)
createsegtracereport('NaTa_t.gNaTa_t_report.h5',gNaTa_T_data.T)

In [ ]:
%cd ..

In [ ]:
#adjust so that a folder is generated for complex and reduced cell and with simulation time as part of the folder name
#maybe include syn distribution in name too